<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
'''
-*- coding:utf-8 -*-
本项目是爬取李宗盛网易云音乐的所有歌曲的歌词，而后实现词云化、关键词提取、词性标注和文本聚类。
'''
import jieba.analyse
import jieba.posseg as pseg
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import jieba
import requests
import sys
import os
from lxml import etree
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

'''
伪造请求头，请求网易云歌曲页面的信息
'''
headers = {
    'Referer': 'http://music.163.com',
    'Host': 'music.163.com',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'User-Agent': 'Chrome/10'
}


def get_song_lyric(headers, lyric_url):
    res = requests.request('GET', lyric_url, headers=headers)
    if 'lrc' in res.json():
        lyric = res.json()['lrc']['lyric']
        new_lyric = re.sub(r'[\d:.[\]]', '', lyric)
        return new_lyric
    else:
        return ''


'''
使用 requests 伪造请求头，获取相关页面数据。
再使用 XPath 解析所获数据，然后将解析得到的数据追加到指定列表中。
'''


def get_songs(artist_id):
    page_url = 'https://music.163.com/artist?id=' + artist_id
    res = requests.request('GET', page_url, headers=headers)
    html = etree.HTML(res.text)
    href_xpath = "//*[@id='hotsong-list']//a/@href"
    name_xpath = "//*[@id='hotsong-list']//a/text()"
    hrefs = html.xpath(href_xpath)
    names = html.xpath(name_xpath)

    song_ids = []
    song_names = []
    for href, name in zip(hrefs, names):
        song_ids.append(href[9:])
        song_names.append(name)
    return song_ids, song_names


# 设置歌手对应 ID，以便于请求对应歌手数据
artist_id = '3683'
[song_ids, song_names] = get_songs(artist_id)

all_word = ''
for (song_id, song_name) in zip(song_ids, song_names):
    # 歌词 API URL
    lyric_url = 'http://music.163.com/api/song/lyric?os=pc&id=' + \
        song_id + '&lv=-1&kv=-1&tv=-1'
    lyric = get_song_lyric(headers, lyric_url)
    all_word = all_word + ' ' + lyric

with open('163_maobuying_musci_lyrics.txt', 'w', encoding="utf-8") as f:
    f.write(all_word)


'''
停用词使用哈工大停用词表，若有额外停用词，可再额外添加
'''


def remove_stop_words(f):
    with open("../../../Data Science/stopwords-master/哈工大停用词表.txt", encoding="utf-8") as fn:
        stop_words = fn.readlines()
    stop_words_addition = ["维度"]
    stop_words = list(stop_words)
    stop_words.extend(stop_words_addition)
    for stop_word in stop_words:
        f = f.replace(stop_word, "")
    return f


'''
使用 jieba 库的 analyse 模块提取关键词：
提取数量 topK：10；
提取关键词的词性：'ns', 'n', 'vn', 'v', 'nr'
'''


def select_key_words(f):
    words = pseg.cut(f.strip())
    tags_pairs = jieba.analyse.extract_tags(f,
                                            topK=10,
                                            withWeight=True,
                                            allowPOS=['ns', 'n',
                                                      'vn', 'v', 'nr'],
                                            withFlag=True)
    tags_list = [(i[0].word, i[0].flag, i[1]) for i in tags_pairs]
    tags_pd = pd.DataFrame(tags_list, columns=['word', 'flags', 'weight'])
    print("排名前十的关键词及其词性、权重为：")
    print(tags_pd)


'''
使用 jieba 库的 posseg 模块标注词性
'''


def words_POS_tagging(f):
    words = pseg.cut(f)
    words_pd = pd.DataFrame(words, columns=["word", "type"])
    words_gb = words_pd.groupby(
        ['type'])['word'].count().sort_values(ascending=False)
    words_pd_index = words_pd["type"].isin(['ns'])
    words_pd_select = words_pd[words_pd_index]
    print("各类词及其数量如下：")
    print(words_gb)


'''
使用 jieba 库分词，而后使用 wordcloud 库中的 WordCloud 模块词云化
词云化时的参数：
设置字体路径，否则无法正常显示中文；
设置最大词数为 100；
设置词云的尺寸宽×高：2000px × 1200px；
设置 collocation 为 True，避免词中出现重复词
'''


def create_word_cloud(f):
    cut_text = " ".join(jieba.cut(f, cut_all=False, HMM=True))
    wc = WordCloud(font_path="C:\Windows\Fonts\simhei.ttf",
                   max_words=100,
                   width=2000,
                   height=1200,
                   collocations=False)
    word_cloud = wc.generate(cut_text)
    picture_path = "./new_output/wordcloud_wang_yi_yun.jpg"
    word_cloud.to_file(picture_path)
    plt.imshow(word_cloud)
    plt.axis("off")
    print("开始生成词云：")
    plt.show()

'''
基本思路是：
1. 使用 jieba 库中 posseg 模块分割数据
2. 只选取某些词性的词
3. 计算这些词的 TF-IDF 值
4. 将 TF-IDF 转换为空间距离
5. 使用 KMeans 算法对这些词进行聚类
6. 显示聚类结果
'''
def word_HanLP(f):
    seg_list = pseg.cut(f)
    word_list = [i.word for i in seg_list if i.flag in ['a', 'ag', 'an']]
    vectorizer = TfidfVectorizer(use_idf=True)
    X = vectorizer.fit_transform(word_list)

    model_kmeans = KMeans(n_clusters=3)
    model_kmeans.fit(X)

    cluster_labels = model_kmeans.labels_
    word_vectors = vectorizer.get_feature_names()
    word_value = X.toarray()

    f_matrix = np.hstack(word_value,
                         cluster_labels.reshape(word_value.shape[0], 1))
    word_vectors.append('cluster_labels')
    comment_pd = pd.DataFrame(comment_matrix,
                              columns=word_vectors)
    word_importance = np.sum(comment_cluster1, axis=0)
    print(word_importance)


all_word = remove_stop_words(all_word)
# select_key_words(all_word)
# words_POS_tagging(all_word)
create_word_cloud(all_word)
word_HanLP(all_word)

本项目存在一定缺陷，如下：
数据获取方面：由于只在网易云音乐获取数据，因此可能出现歌曲信息获取不完整的情况。如果要解决这个问题，其实第一步要做的事，是把李宗盛所有歌曲做个调查，然后再对比所爬取的歌曲信息，看是否存在数据缺漏问题。<br>还有个折中法子，就是把李宗盛唱片销售量排前几的歌曲全部下载下来，然后再分析，即取代表性歌曲做分析。这里采取的思路也是这样，不过是爬取网易云李宗盛前 50 首热门歌曲信息。

数据清洗：实际上，前 50 首歌曲中，很有可能出现重复歌曲，只是版本不一样。因此，这就出现数据重复采集。如有可能，首先应该观察要爬取的歌曲数据，然后再选择是否跳过某些歌曲。